In [58]:
from datasets import load_dataset
from sentence_transformers.losses import CosineSimilarityLoss

from setfit import SetFitModel, SetFitTrainer
import evaluate
import numpy as np

In [59]:
import os
print(os.getcwd())

/mnt/c/Users/leonl/OneDrive/College/Senior/CSE 481DS/Analysis


In [60]:
def to_bool(string:str) -> bool:
    string = str(string).strip().casefold()
    if string == 'nan':
        return False
    if string in ('true', 'yes', '1'):
        return True
    if string in ('false', 'no', '0', 'none'):
        return False
    return False

def dsmap(ds, col, fn):
    # I hate huggingface datasets
    return ds.map(lambda x: {**x, **{col: fn(x[col])}})

In [85]:
# Load trainset from ../irr/train_set_gpt/caitlyn.csv
trainset = load_dataset("csv", data_files="./classify/irr/test_set_gpt/caitlyn.csv")["train"]
testset  = load_dataset("csv", data_files="./classify/irr/train_set_human/leon.csv")["train"]

# Set all columns to bool
trainset = dsmap(trainset, 'BreachMentioned', lambda x: int(to_bool(x)))
trainset = dsmap(trainset, 'CompanyMentioned', lambda x: int(bool(x)))
testset  = dsmap(testset, 'BreachMentioned', lambda x: int(to_bool(x)))
testset  = dsmap(testset, 'CompanyMentioned', lambda x: int(bool(x)))

# Remove non-input, non-target columns
trainset = trainset.remove_columns(['Date', 'Publication', 'URL'] + [x for x in trainset.column_names if x.startswith('Unnamed: ')])
testset  = testset.remove_columns(['Date', 'Publication', 'URL'] + [x for x in testset.column_names if x.startswith('Unnamed: ')])

# Split into BreachMentioned and CompanyMentioned
trainset_breach = trainset.remove_columns(['CompanyMentioned'])
testset_breach  = testset.remove_columns(['CompanyMentioned'])

trainset_company = trainset.remove_columns(['BreachMentioned'])
testset_company  = testset.remove_columns(['BreachMentioned'])

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

In [62]:
model_breach  = SetFitModel.from_pretrained("sentence-transformers/paraphrase-mpnet-base-v2")
model_company = SetFitModel.from_pretrained("sentence-transformers/paraphrase-mpnet-base-v2")

model_head.pkl not found on HuggingFace Hub, initialising classification head with random weights. You should TRAIN this model on a downstream task to use it for predictions and inference.
model_head.pkl not found on HuggingFace Hub, initialising classification head with random weights. You should TRAIN this model on a downstream task to use it for predictions and inference.


In [63]:
trainset_breach[100]

{'Headline': 'Bitcoin vaults to new record above $4K, boosted by Japan and multiplying its value fourfold',
 'BreachMentioned': 0}

In [64]:
# Train setfit
trainer = SetFitTrainer(
    model=model_breach,
    train_dataset=trainset_breach,
    eval_dataset=testset_breach,
    loss_class=CosineSimilarityLoss,
    batch_size=16,
    num_iterations=20, # Number of text pairs to generate for contrastive learning
    num_epochs=1, # Number of epochs to use for contrastive learning
    column_mapping={"Headline": "text", "BreachMentioned": "label"}
)

trainer.train()

os.makedirs('./classify/setfit/model_breach', exist_ok=True)
model_breach._save_pretrained('./classify/setfit/model_breach')

metrics = trainer.evaluate()
print(metrics)

/tmp/ipykernel_3727/3804403911.py:2: DeprecationWarning: `SetFitTrainer` has been deprecated and will be removed in v2.0.0 of SetFit. Please use `Trainer` instead.
  trainer = SetFitTrainer(
Applying column mapping to the training dataset
Applying column mapping to the evaluation dataset


***** Running training *****
  Num unique pairs = 8000
  Batch size = 16
  Num epochs = 1


Step,Training Loss
1,0.601100
50,0.288000
100,0.057700
150,0.003700
200,0.001000
250,0.000800
300,0.000400
350,0.000300
400,0.000500
450,0.000400


Computing widget examples:   0%|          | 0/1 [00:00<?, ?example/s]

***** Running evaluation *****


{'accuracy': 0.96}


In [80]:
print(f"Number of positive pairs: {len(testset_company.get_positive_pairs())}")
print(f"Number of negative pairs: {len(testset_company.get_negative_pairs())}")

AttributeError: 'Dataset' object has no attribute 'get_positive_pairs'

In [86]:
# Assert that we have both 1 and 0 labels in both
assert 1 in testset_company['CompanyMentioned']

In [87]:
# Train CompanyMentioned model
# Train setfit
trainer = SetFitTrainer(
    model=model_company,
    train_dataset=trainset_company,
    eval_dataset=testset_company,
    loss_class=CosineSimilarityLoss,
    batch_size=16,
    num_iterations=20, # Number of text pairs to generate for contrastive learning
    num_epochs=1, # Number of epochs to use for contrastive learning
    column_mapping={"Headline": "text", "CompanyMentioned": "label"}
)

trainer.train()

os.makedirs('./classify/setfit/model_company', exist_ok=True)
model_breach._save_pretrained('./classify/setfit/model_company')

metrics = trainer.evaluate()
print(metrics)

/tmp/ipykernel_3727/3880213943.py:3: DeprecationWarning: `SetFitTrainer` has been deprecated and will be removed in v2.0.0 of SetFit. Please use `Trainer` instead.
  trainer = SetFitTrainer(
Applying column mapping to the training dataset
Applying column mapping to the evaluation dataset


Map:   0%|          | 0/200 [00:00<?, ? examples/s]

***** Running training *****
  Num unique pairs = 8000
  Batch size = 16
  Num epochs = 1


Step,Training Loss
1,0.666800
50,0.305400
100,0.184300
150,0.027700
200,0.001500
250,0.000600
300,0.000400
350,0.000400
400,0.000400
450,0.000400


Computing widget examples:   0%|          | 0/1 [00:00<?, ?example/s]

***** Running evaluation *****


{'accuracy': 0.725}


In [ ]:
# saved_model = SetFitModel._from_pretrained('./classify/setfit/model')

: 

: 

: 

: 